# Convert list of values and dates into dekads

In [ ]:
import pandas as pd
import numpy as np

begin = pd.datetime(2002,7,1)
end = pd.datetime(2016,12,31)

dtrange = pd.date_range(begin, end)

p1 = np.random.rand(len(dtrange)) + 5
p2 = np.random.rand(len(dtrange)) + 10

df = pd.DataFrame({'p1': p1, 'p2': p2}, index=dtrange)

In [ ]:
type(p1)

In [ ]:
d = df.index.day - np.clip((df.index.day-1) // 10, 0, 2)*10 - 1
date = df.index.values - np.array(d, dtype="timedelta64[D]")
df.groupby(date).mean()

In [ ]:
date

In [ ]:
df.index.dayofyear

# Resample daily files in folders to dekads

In [1]:
import rasterio
import numpy as np
from glob import glob
import os
from rasterio.plot import show

In [2]:
def read_file(file):
    with rasterio.open(file) as src:
        return(src.read())
    
def dekad_resample(root_dir, out_dir, var):
    folds = [fold for fold in sorted(os.listdir(root_dir))]
    for fold in folds:
        os.chdir(root_dir + fold + '/')
        files = [file for file in sorted(os.listdir(os.getcwd()))]
        dekad1 = [read_file(x) for x in files[0:10]]
        dekad2 = [read_file(x) for x in files[10:20]]
        dekad3 = [read_file(x) for x in files[20:]]
        
     # Perform averaging
        dekad1_out = np.mean(dekad1, axis=0)
        dekad2_out = np.mean(dekad2, axis=0)
        dekad3_out = np.mean(dekad3, axis=0)

        with rasterio.open(files[0]) as src:
            meta = src.meta

        meta.update(dtype=rasterio.float32)

        # Write output file
        with rasterio.open(out_dir + var + fold + '_01.tif', 'w', **meta) as dst:
            dst.write(dekad1_out.astype(rasterio.float32))

        with rasterio.open(out_dir + var + fold + '_02.tif', 'w', **meta) as dst:
            dst.write(dekad2_out.astype(rasterio.float32))

        with rasterio.open(out_dir + var + fold + '_03.tif', 'w', **meta) as dst:
            dst.write(dekad3_out.astype(rasterio.float32))


In [3]:
tmax_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Months/' 
out_tmax_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/'


In [ ]:
sm_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Months/' 
out_sm_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/' 

In [4]:
dekad_resample(tmax_dir, out_tmax_dir, var = 'Tmax_')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
